In [26]:
%%capture
!pip install langchain-iris

!pip install testcontainers-iris \
                fastembed \
                openai \
                tiktoken \
                python-dotenv


We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [27]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


In [3]:
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.fastembed import FastEmbedEmbeddings

from langchain_iris import IRISVector


In [5]:
loader = TextLoader("state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()
embeddings = FastEmbedEmbeddings()

In [6]:
import time
import os
from testcontainers.iris import IRISContainer

# Enterprise version with iris.key placed in the user's home directory
# license_key = os.path.abspath(os.path.expanduser("~/iris.key"))
# image = 'containers.intersystems.com/intersystems/iris:2023.3'

# Community Edition
license_key = None
image = 'containers.intersystems.com/intersystems/iris-community:2023.3'

# image = 'iris:2024.1'
container = IRISContainer(image, username="demo", password="demo", namespace="demo", license_key=license_key)
container.with_exposed_ports(1972, 52773)
container.start()
CONNECTION_STRING = container.get_connection_url()

time.sleep(1)
print('Started')


Pulling image containers.intersystems.com/intersystems/iris:2023.3
Container started: bd7acb4e7e13
Waiting to be ready...


Started


In [7]:
COLLECTION_NAME = "state_of_the_union_test"

db = IRISVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [12]:
ids=db.get()["ids"]
db.delete(ids)

In [13]:
query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query)

In [14]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

In [34]:
db.add_documents([Document(page_content="foo")])
docs_with_score = db.similarity_search_with_score("foo")
docs_with_score[0]

(Document(page_content='foo'), 0.0)

In [35]:
docs_with_score

[(Document(page_content='foo'), 0.0),
 (Document(page_content='One was stationed at bases and breathing in toxic smoke from “burn pits” that incinerated wastes of war—medical and hazard material, jet fuel, and more. \n\nWhen they came home, many of the world’s fittest and best trained warriors were never the same. \n\nHeadaches. Numbness. Dizziness. \n\nA cancer that would put them in a flag-draped coffin. \n\nI know. \n\nOne of those soldiers was my son Major Beau Biden. \n\nWe don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. \n\nBut I’m committed to finding out everything we can. \n\nCommitted to military families like Danielle Robinson from Ohio. \n\nThe widow of Sergeant First Class Heath Robinson.  \n\nHe was born a soldier. Army National Guard. Combat medic in Kosovo and Iraq. \n\nStationed near Baghdad, just yards from burn pits the size of football fields. \n\nHeath’s widow Danielle is here with us tonight. They love

In [38]:
retriever = db.as_retriever()
print(retriever)

tags=['IRISVector'] vectorstore=<langchain_iris.vectorstores.IRISVector object at 0x16984e390>


In [37]:
container.stop()